# 필요한 라이브러리

In [1]:
import glob  #glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다
import tensorflow as tf
import os, re

# 데이터 읽어오기

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*' #os.getenv(x)함수는 환경 변수x의 값을 포함하는 문자열 변수를 반환합니다. txt_file_path 에 "/root/aiffel/lyricist/data/lyrics/*" 저장

txt_list = glob.glob(txt_file_path) #txt_file_path 경로에 있는 모든 파일명을 리스트 형식으로 txt_list 에 할당

raw_corpus = [] 

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() #read() : 파일 전체의 내용을 하나의 문자열로 읽어온다. , splitlines()  : 여러라인으로 구분되어 있는 문자열을 한라인씩 분리하여 리스트로 반환
        raw_corpus.extend(raw) # extend() : 리스트함수로 추가적인 내용을 연장 한다.

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


# 데이터 정제
###### 앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요! preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다. 추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 너무 긴 문장은 노래 가사 작사하기에 어울리지 않을 수도 있겠죠. 그래서 이번에는 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기 를 권합니다.

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [4]:
# 여기에 정제된 문장을 모으기
corpus=[]

for sentence in raw_corpus:
    #내가 원치 않는 문장 건너뛰기
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    if len(sentence) >= 15 : continue
        
    #앞서 구현한 preprocess_sentence()함수를 이용하여 문장을 정제를 하고 담아주자
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
#정제된 결과를 10개만 확인해보자
corpus[:10]

['<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> she tied you <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> in every word <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>']

# 평가 데이터셋 분리
###### 훈련 데이터와 평가 데이터를 분리하세요! tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상 으로 설정하세요! 총 데이터의 20% 를 평가 데이터셋으로 사용해 주세요!

In [5]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # tokenizer.fit_on_texts(texts): 문자 데이터를 입력받아 리스트의 형태로 변환하는 메서드
    tokenizer.fit_on_texts(corpus)
    
    # tokenizer.texts_to_sequences(texts): 텍스트 안의 단어들을 숫자의 시퀀스 형태로 변환하는 메서드 
    tensor = tokenizer.texts_to_sequences(corpus)
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2 143   3 ...   0   0   0]
 [  2 143   3 ...   0   0   0]
 [  2  55 910 ...   0   0   0]
 ...
 [  2 193 570 ...   0   0   0]
 [  2  62  22 ...   0   0   0]
 [  2  32  16 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7fd3fae227f0>


In [6]:
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성
tgt_input = tensor[:, 1:]    
src_input, tgt_input

(array([[  2, 143,   3, ...,   0,   0,   0],
        [  2, 143,   3, ...,   0,   0,   0],
        [  2,  55, 910, ...,   0,   0,   0],
        ...,
        [  2, 193, 570, ...,   0,   0,   0],
        [  2,  62,  22, ...,   0,   0,   0],
        [  2,  32,  16, ...,   0,   0,   0]], dtype=int32),
 array([[143,   3,   0, ...,   0,   0,   0],
        [143,   3,   0, ...,   0,   0,   0],
        [ 55, 910,   8, ...,   0,   0,   0],
        ...,
        [193, 570,  11, ...,   0,   0,   0],
        [ 62,  22,  33, ...,   0,   0,   0],
        [ 32,  16,  11, ...,   0,   0,   0]], dtype=int32))

In [7]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=20)

# 인공지능 만들기

In [9]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        return out
embedding_size = 500 
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [10]:
loss = tf.keras.losses.SparseCategoricalCrossentropy( 
    from_logits=True, reduction='none') # 클래스 분류 문제에서 softmax 함수를 거치면 from_logits = False(default값),그렇지 않으면 from_logits = True.
optimizer = tf.keras.optimizers.Adam() 

# 모델을 학습시키키 위한 학습과정을 설정하는 단계이다.
model.compile(loss=loss, optimizer=optimizer) # 손실함수와 훈련과정을 설정했다.
model.fit(src_input,tgt_input,validation_data=(enc_val,dec_val),epochs=7) # 만들어둔 데이터셋으로 모델을 학습한다. 7번 학습을 반복하겠다는 의미다.

Epoch 1/7
380/380 [==============================] - 75s 91ms/step - loss: 1.5486 - val_loss: 1.2756
Epoch 2/7
380/380 [==============================] - 36s 95ms/step - loss: 1.2234 - val_loss: 1.1397
Epoch 3/7
380/380 [==============================] - 36s 94ms/step - loss: 1.0962 - val_loss: 1.0145
Epoch 4/7
380/380 [==============================] - 36s 93ms/step - loss: 0.9926 - val_loss: 0.9266
Epoch 5/7
380/380 [==============================] - 36s 95ms/step - loss: 0.9176 - val_loss: 0.8640
Epoch 6/7
380/380 [==============================] - 36s 94ms/step - loss: 0.8623 - val_loss: 0.8116
Epoch 7/7
380/380 [==============================] - 36s 95ms/step - loss: 0.8229 - val_loss: 0.7753


In [11]:
#문장생성 함수 정의
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20): 
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence]) #텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듦
    while True: #루프를 돌면서 init_sentence에 단어를 하나씩 생성
        # 1.입력받은 문장의 텐서를 입력
        predict = model(test_tensor) 
        # 2.예측된 값 중 가장 높은 확률인 word index를 뽑아냄
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3. 2.에서 예측된 word index를 문장 뒤에 붙임
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4. 모델이 <end>를 예측했거나, max_len에 도달하면 문장 생성을 마침 / 도달 하지 못하면 while 루프를 돌면서 다음 단어를 예측함.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 문장을 반환

In [14]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

- 이번 프로젝트에서 **어려웠던 점,** : 전체적으로 다 어려웠습니다
- 프로젝트를 진행하면서 **알아낸 점** 혹은 **아직 모호한 점** : 어떤식으로 돌아가는지 알았어요
- 루브릭 평가 지표를 맞추기 위해 **시도한 것들** : 잘 모르겠습니다
- 만약에 루브릭 평가 관련 지표를 **달성 하지 못했을 때, 이유에 관한 추정**. : 잘 모르겠습니다
- **자기 다짐** : 뭐가 뭔지 잘 모르겠다.. 아직 많이 부족하니 더 봐보자